In [4]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.easter import easter

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention
from tensorflow.keras.layers import LayerNormalization, Dropout, Add
from tensorflow.keras.models import Model

# ---------------------------------------------------------------------
# 0. Reproducibility
# ---------------------------------------------------------------------
tf.random.set_seed(42)
np.random.seed(42)

# ---------------------------------------------------------------------
# 1. Load data for user_id = 10
# ---------------------------------------------------------------------
df = pd.read_csv(
    r"C:/Users/loics/OneDrive/Documents/1. BAM/BLOCK 5/Assignment coding/synthetic_transactions_daily_enriched.csv",
    parse_dates=["date"],
)
user_df = df[df["user_id"] == 10].sort_values("date").reset_index(drop=True)

# ---------------------------------------------------------------------
# 2. Calendar + holiday features
# ---------------------------------------------------------------------
user_df["day_of_week"]       = user_df["date"].dt.weekday   # 0 = Monday
user_df["month"]             = user_df["date"].dt.month
user_df["day_of_month"]      = user_df["date"].dt.day
user_df["is_first_of_month"] = (user_df["day_of_month"] == 1).astype(int)

def get_nl_holidays(start_year, end_year):
    hol = set()
    for yr in range(start_year, end_year + 1):
        hol.add(date(yr, 1, 1))          # New Year
        e = easter(yr)
        hol |= {e - timedelta(2), e + timedelta(1)}           # Good Fri / Easter Mon
        hol |= {e + timedelta(39), e + timedelta(50)}         # Ascension / Whit Mon
        kday = date(yr, 4, 26 if date(yr, 4, 27).weekday() == 6 else 27)
        hol.add(kday)                                         # King's Day
        if yr == 2025: hol.add(date(yr, 5, 5))                # Liberation
        hol |= {date(yr, 12, 25), date(yr, 12, 26)}           # Christmas
    return hol

holidays = get_nl_holidays(2022, 2025)
user_df["holiday"] = user_df["date"].dt.date.isin(holidays).astype(int)

# ---------------------------------------------------------------------
# 3. Forecast horizon & window sizes
# ---------------------------------------------------------------------
forecast_start = pd.Timestamp("2025-03-01")
forecast_end   = pd.Timestamp("2025-05-31")
H = (forecast_end - forecast_start).days + 1   # 92 days
L = 365                                        # 1-year history window

train_df = user_df[user_df["date"] < forecast_start].copy()

# ---------------------------------------------------------------------
# 4. Prepare numeric series
# ---------------------------------------------------------------------
income_series  = train_df["income"].values                     # ≥ 0
expense_series = train_df["expenses"].values                   # ≤ 0

num_train = len(train_df)

# one-hot weekday & month
dow = np.zeros((num_train, 7),  dtype=np.float32)
mon = np.zeros((num_train, 12), dtype=np.float32)
dow[np.arange(num_train), train_df["day_of_week"]] = 1.0
mon[np.arange(num_train), train_df["month"] - 1]   = 1.0

first_of_month = train_df["is_first_of_month"].astype(np.float32).values.reshape(-1, 1)
holiday_flag   = train_df["holiday"].astype(np.float32).values.reshape(-1, 1)

# ---------------------------------------------------------------------
# 5. Train / validation window indices
# ---------------------------------------------------------------------
train_idx, val_idx = [], []
for s in range(num_train - L - H + 1):
    end_date = train_df.iloc[s + L + H - 1]["date"]
    (train_idx if end_date <= pd.Timestamp("2024-12-31") else val_idx).append(s)
train_idx, val_idx = np.array(train_idx), np.array(val_idx)

# ---------------------------------------------------------------------
# 6. Scaling
# ---------------------------------------------------------------------
income_max       = income_series.max() or 1.0             # avoid div/0
expense_min_abs  = abs(expense_series.min()) or 1.0       # most negative magnitude

income_scaled  = income_series  / income_max              # [0, 1]
expense_scaled = expense_series / (-expense_min_abs)      # [-1, 0]

# ---------------------------------------------------------------------
# 7. Dataset builder
# ---------------------------------------------------------------------
def make_xy(idxs):
    Xe, Xd = [], []
    y_inc_flag, y_inc_amt, y_exp = [], [], []
    for i in idxs:
        hist = slice(i,      i + L)
        fut  = slice(i + L,  i + L + H)

        Xe.append(np.hstack([
            income_scaled[hist].reshape(L,1),
            expense_scaled[hist].reshape(L,1),
            dow[hist], mon[hist], first_of_month[hist], holiday_flag[hist]
        ]))
        Xd.append(np.hstack([
            dow[fut], mon[fut], first_of_month[fut], holiday_flag[fut]
        ]))

        # targets
        inc_fut_scaled = income_scaled[fut]
        y_inc_flag.append((inc_fut_scaled > 0).astype(np.float32))
        y_inc_amt.append(inc_fut_scaled)
        y_exp.append(expense_scaled[fut])
    return (np.array(Xe, dtype=np.float32),
            np.array(Xd, dtype=np.float32),
            np.array(y_inc_flag, dtype=np.float32),
            np.array(y_inc_amt,  dtype=np.float32),
            np.array(y_exp,      dtype=np.float32))

Xe_tr, Xd_tr, y_flag_tr, y_inc_tr, y_exp_tr = make_xy(train_idx)
Xe_val, Xd_val, y_flag_val, y_inc_val, y_exp_val = (
    make_xy(val_idx) if len(val_idx) else (None,)*5
)

# ---------------------------------------------------------------------
# 8. Loss functions
# ---------------------------------------------------------------------
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)

quantiles = tf.constant([0.1, 0.5, 0.9], dtype=tf.float32)
Q = len(quantiles)

def masked_pinball_loss(y_true, y_pred):
    """
    Pinball loss on positive-income days only (y_true==0 ⇒ no loss).
    """
    # y_true / y_pred shape = (B, H, Q)
    mask = tf.cast(tf.greater(y_true[..., 0], 0), tf.float32)  # shape (B, H)
    e = y_true - y_pred                                        # (B,H,Q)
    q = quantiles[None, None, :]                               # broadcast
    loss = tf.maximum(q*e, (q-1)*e)                            # pinball
    loss = loss * mask[..., None]                              # zero out on no-income days
    return tf.reduce_mean(loss)

def pinball_loss(y_true, y_pred):
    """
    Standard pinball for expenses (always negative, but we don't mask).
    """
    e = y_true - y_pred
    q = quantiles[None, None, :]
    return tf.reduce_mean(tf.maximum(q*e, (q-1)*e))

# ---------------------------------------------------------------------
# 9. Transformer factory
# ---------------------------------------------------------------------
def build_transformer(L, H, enc_dim, dec_dim,
                      d_model=64, n_heads=4,
                      ff_dim=256, do_rate=0.1, n_layers=2,
                      last_activation="linear", output_units=1):
    enc_in = Input((L,  enc_dim))
    dec_in = Input((H,  dec_dim))

    # sinusoidal positional encoding
    def pos_enc(T):
        pos = np.arange(T)[:, None]
        i   = np.arange(d_model)[None, :]
        angle = pos / (10000 ** (2*(i//2)/d_model))
        pe = np.zeros((T, d_model), dtype=np.float32)
        pe[:, 0::2] = np.sin(angle[:, 0::2])
        pe[:, 1::2] = np.cos(angle[:, 1::2])
        return tf.constant(pe)

    x_enc = Dense(d_model)(enc_in) + pos_enc(L)
    x_dec = Dense(d_model)(dec_in) + pos_enc(H)

    for _ in range(n_layers):
        # encoder
        attn = MultiHeadAttention(num_heads=n_heads, key_dim=d_model,
                                  dropout=do_rate)(x_enc, x_enc)
        x_enc = LayerNormalization(epsilon=1e-6)(Add()([x_enc, Dropout(do_rate)(attn)]))
        ff = Dense(ff_dim, 'relu')(x_enc); ff = Dense(d_model)(ff)
        x_enc = LayerNormalization(epsilon=1e-6)(Add()([x_enc, Dropout(do_rate)(ff)]))
        # decoder
        attn = MultiHeadAttention(num_heads=n_heads, key_dim=d_model,
                                  dropout=do_rate)(x_dec, x_dec,
                                                   use_causal_mask=True)
        x_dec = LayerNormalization(epsilon=1e-6)(Add()([x_dec, Dropout(do_rate)(attn)]))
        cross = MultiHeadAttention(num_heads=n_heads, key_dim=d_model,
                                   dropout=do_rate)(x_dec, x_enc, x_enc)
        x_dec = LayerNormalization(epsilon=1e-6)(Add()([x_dec, Dropout(do_rate)(cross)]))
        ff = Dense(ff_dim, 'relu')(x_dec); ff = Dense(d_model)(ff)
        x_dec = LayerNormalization(epsilon=1e-6)(Add()([x_dec, Dropout(do_rate)(ff)]))

    out = Dense(output_units, activation=last_activation)(x_dec)
    return Model([enc_in, dec_in], out)

enc_dim = Xe_tr.shape[2]
dec_dim = Xd_tr.shape[2]

# ---------------------------------------------------------------------
# 10. Stage-1: Income CLASSIFIER
# ---------------------------------------------------------------------
clf_model = build_transformer(L, H, enc_dim, dec_dim,
                              last_activation="sigmoid", output_units=1)

clf_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3, clipnorm=1.0),
                  loss=bce,
                  metrics=["accuracy"])

cb_clf = [tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True)] if Xe_val is not None else []

clf_model.fit([Xe_tr, Xd_tr], y_flag_tr[..., None],   # add channel dim
              validation_data=([Xe_val, Xd_val], y_flag_val[..., None])
              if Xe_val is not None else None,
              epochs=50, batch_size=32, shuffle=True,
              callbacks=cb_clf, verbose=2)

# ---------------------------------------------------------------------
# 11. Stage-2: Income REGRESSOR (masked pinball)
# ---------------------------------------------------------------------
y_inc_tr_exp  = np.repeat(y_inc_tr[..., None], Q, axis=2)
y_inc_val_exp = np.repeat(y_inc_val[..., None], Q, axis=2) if Xe_val is not None else None

reg_model = build_transformer(L, H, enc_dim, dec_dim,
                              last_activation="linear", output_units=Q)

reg_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3, clipnorm=1.0),
                  loss=masked_pinball_loss)

cb_reg = [tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True)] if Xe_val is not None else []

reg_model.fit([Xe_tr, Xd_tr], y_inc_tr_exp,
              validation_data=([Xe_val, Xd_val], y_inc_val_exp)
              if Xe_val is not None else None,
              epochs=50, batch_size=32, shuffle=True,
              callbacks=cb_reg, verbose=2)

# ---------------------------------------------------------------------
# 12. Expense model
# ---------------------------------------------------------------------
y_exp_tr_exp  = np.repeat(y_exp_tr[..., None], Q, axis=2)
y_exp_val_exp = np.repeat(y_exp_val[..., None], Q, axis=2) if Xe_val is not None else None

exp_model = build_transformer(L, H, enc_dim, dec_dim,
                              last_activation="linear", output_units=Q)

exp_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3, clipnorm=1.0),
                  loss=pinball_loss)

cb_exp = [tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True)] if Xe_val is not None else []

exp_model.fit([Xe_tr, Xd_tr], y_exp_tr_exp,
              validation_data=([Xe_val, Xd_val], y_exp_val_exp)
              if Xe_val is not None else None,
              epochs=50, batch_size=32, shuffle=True,
              callbacks=cb_exp, verbose=2)

# ---------------------------------------------------------------------
# 13. Build encoder / decoder inputs for forecast
# ---------------------------------------------------------------------
def enc_block(series_scaled):
    return np.hstack([
        series_scaled[-L:].reshape(L,1),
        expense_scaled[-L:].reshape(L,1),
        dow[-L:], mon[-L:], first_of_month[-L:], holiday_flag[-L:]
    ])[None, ...]

enc_input = enc_block(income_scaled)          # ← same for all three models
dec_dow   = np.zeros((H, 7),  dtype=np.float32)
dec_mon   = np.zeros((H, 12), dtype=np.float32)

future_dates = pd.date_range(forecast_start, forecast_end, freq="D")
dec_dow[np.arange(H), future_dates.weekday] = 1.0
dec_mon[np.arange(H), future_dates.month-1] = 1.0
dec_first  = (future_dates.day == 1).astype(np.float32).reshape(-1,1)
dec_hol    = np.array([d.date() in holidays for d in future_dates], np.float32).reshape(-1,1)

dec_input = np.hstack([dec_dow, dec_mon, dec_first, dec_hol])[None, ...]

# ---------------------------------------------------------------------
# 14. Forecast
# ---------------------------------------------------------------------
prob_salary  = clf_model.predict([enc_input, dec_input])[0].squeeze()      # (H,)
inc_amount_s = reg_model.predict([enc_input, dec_input])[0]                # (H,Q)
exp_pred_s   = exp_model.predict([enc_input, dec_input])[0]                # (H,Q)

# Gate income amounts: prob > 0.5 ⇒ keep; else set to 0
gate = (prob_salary > 0.5).astype(np.float32)[:, None]     # (H,1)
inc_amount_s *= gate

# Inverse scaling
inc_amount = inc_amount_s * income_max                     # ≥ 0
exp_amount = exp_pred_s  * (-expense_min_abs)              # ≤ 0

# Ensure no positive expenses
exp_amount = np.where(exp_amount > 0, 0, exp_amount)

# ---------------------------------------------------------------------
# 15. Results dataframe
# ---------------------------------------------------------------------
pred_df = pd.DataFrame({
    "date":           future_dates.date,
    "income_prob":    prob_salary,
    "income_p10":     inc_amount[:,0],
    "income_median":  inc_amount[:,1],
    "income_p90":     inc_amount[:,2],
    "expense_p10":    exp_amount[:,0],
    "expense_median": exp_amount[:,1],
    "expense_p90":    exp_amount[:,2],
})

print(pred_df.head())



Epoch 1/50
20/20 - 31s - loss: 0.2357 - accuracy: 0.9209 - val_loss: 0.1673 - val_accuracy: 0.9565 - 31s/epoch - 2s/step
Epoch 2/50
20/20 - 24s - loss: 0.1294 - accuracy: 0.9632 - val_loss: 0.0910 - val_accuracy: 0.9707 - 24s/epoch - 1s/step
Epoch 3/50
20/20 - 23s - loss: 0.0441 - accuracy: 0.9893 - val_loss: 0.0434 - val_accuracy: 0.9891 - 23s/epoch - 1s/step
Epoch 4/50
20/20 - 24s - loss: 0.0222 - accuracy: 0.9950 - val_loss: 0.0305 - val_accuracy: 0.9854 - 24s/epoch - 1s/step
Epoch 5/50
20/20 - 23s - loss: 0.0181 - accuracy: 0.9953 - val_loss: 0.0396 - val_accuracy: 0.9812 - 23s/epoch - 1s/step
Epoch 6/50
20/20 - 23s - loss: 0.0154 - accuracy: 0.9958 - val_loss: 0.0416 - val_accuracy: 0.9842 - 23s/epoch - 1s/step
Epoch 7/50
20/20 - 25s - loss: 0.0138 - accuracy: 0.9959 - val_loss: 0.0486 - val_accuracy: 0.9871 - 25s/epoch - 1s/step
Epoch 8/50
20/20 - 23s - loss: 0.0122 - accuracy: 0.9964 - val_loss: 0.0393 - val_accuracy: 0.9854 - 23s/epoch - 1s/step
Epoch 9/50
20/20 - 24s - loss: 0

In [5]:
from pathlib import Path
CSV_PATH = Path(r"C:/Users/loics/OneDrive/Documents/1. BAM/BLOCK 5/Assignment coding/synthetic_transactions_daily_enriched.csv")
OUT_PATH = CSV_PATH.parent / f"final_income_expense_predictions_v2.csv"
pd.DataFrame(pred_df).to_csv(OUT_PATH, index=False)
print(f"✔  Predictions saved → {OUT_PATH}")

✔  Predictions saved → C:\Users\loics\OneDrive\Documents\1. BAM\BLOCK 5\Assignment coding\final_income_expense_predictions_v2.csv
